### webscraper to download trip data

In [2]:
# import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [3]:
# initialize browser instance
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [43]:
# visit url
url = 'https://s3.amazonaws.com/tripdata/index.html'
browser.visit(url)

In [5]:
# turn into soup
html = browser.html
soup = bs(html,'html.parser')

In [21]:
# find all table data elements
data = soup.find_all('td')

In [23]:
# narrow down available download links
links = []
for item in data:
    if item.a:
        if 'JC' not in item.a['href']:
            links.append(item.a['href'])

In [41]:
links[0].split('/')[-1]

'201306-citibike-tripdata.zip'

In [44]:
# visit each source for download, ignore two files
for i, link in enumerate(links):
    if 'index' not in link and '201307-201402' not in link:
        browser.click_link_by_partial_text(links[i].split('/')[-1])

In [94]:
# close browser instance
browser.quit()

### unzip and extract CSV files

In [1]:
import os
import zipfile

In [100]:
# change directory to downloads
path = 'C:/Users/nssas/Downloads/'
os.chdir(path)

# unzip each file, move to resources directory
dest = 'D:/tableau_csvs/trip_data/'
for file in os.listdir('.'):
    if 'citibike-tripdata' in file:
        if 'index' not in file and '201307-201402' not in file:
            zip_ref = zipfile.ZipFile(file,'r')
            dis = [p for p in zip_ref.namelist() if p.startswith('20') and p.endswith('.csv')][0]
            zip_ref.extract(dis,dest)
            zip_ref.close()
            
            print(f'Extracted {file}...')

Extracted 201306-citibike-tripdata.zip...
Extracted 201307-citibike-tripdata.zip...
Extracted 201308-citibike-tripdata.zip...
Extracted 201309-citibike-tripdata.zip...
Extracted 201310-citibike-tripdata.zip...
Extracted 201311-citibike-tripdata.zip...
Extracted 201312-citibike-tripdata.zip...
Extracted 201401-citibike-tripdata.zip...
Extracted 201402-citibike-tripdata.zip...
Extracted 201403-citibike-tripdata.zip...
Extracted 201404-citibike-tripdata.zip...


### rename all columns to standard names

In [4]:
import pandas as pd

In [2]:
# navigate back to project directory
nav = 'D:/tableau_csvs/'
os.chdir(nav)

In [15]:
# check if there are new occuring column names
unique = []
columns = []
for file in os.listdir('trip_data/'):
    if file.endswith('.csv'):
        df = pd.read_csv(f'trip_data/{file}',nrows=0,engine='python',error_bad_lines=False)
        cols = df.columns.values.tolist()
        d = []
        for col in cols:
            col = ''.join(col.split(' ')).lower()
            d.append(col)
            if col not in unique:
                unique.append(col)
                print(col)
        columns.append(d)
        print('----')

tripduration
starttime
stoptime
startstationid
startstationname
startstationlatitude
startstationlongitude
endstationid
endstationname
endstationlatitude
endstationlongitude
bikeid
usertype
birthyear
gender
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----


In [23]:
# find if all columns are matching
a = columns[0]
matching = []
for col in columns:
    if col == a:
        matching.append(True)
    else:
        matching.append(False)

In [24]:
# confirm all columns are matching
print(f'Datasets with same columns: {matching.count(True)}')
print(f'Datasets with new columns: {matching.count(False)}')

Datasets with same columns: 67
Datasets with new columns: 0


col ids are all the same, just remove spaces push to lowercase

In [48]:
# find all files
for x in range(2013,2019):
    for file in os.listdir('D:/tableau_csvs/trip_data/'):
        if file.startswith(str(x)):
            print(file)
            
    print('----')

2013-06-citibike-tripdata.csv
2013-07 - Citi Bike trip data.csv
2013-08 - Citi Bike trip data.csv
2013-09 - Citi Bike trip data.csv
2013-10 - Citi Bike trip data.csv
2013-11 - Citi Bike trip data.csv
2013-12 - Citi Bike trip data.csv
----
2014-01 - Citi Bike trip data.csv
2014-02 - Citi Bike trip data.csv
2014-03 - Citi Bike trip data.csv
2014-04 - Citi Bike trip data.csv
2014-05 - Citi Bike trip data.csv
2014-06 - Citi Bike trip data.csv
2014-07 - Citi Bike trip data.csv
2014-08 - Citi Bike trip data.csv
201409-citibike-tripdata.csv
201410-citibike-tripdata.csv
201411-citibike-tripdata.csv
201412-citibike-tripdata.csv
----
201501-citibike-tripdata.csv
201502-citibike-tripdata.csv
201503-citibike-tripdata.csv
201504-citibike-tripdata.csv
201505-citibike-tripdata.csv
201506-citibike-tripdata.csv
201507-citibike-tripdata.csv
201508-citibike-tripdata.csv
201509-citibike-tripdata.csv
201510-citibike-tripdata.csv
201511-citibike-tripdata.csv
201512-citibike-tripdata.csv
----
201601-citibike

In [51]:
# replace columns and create annual CSVs
for x in range(2013,2019):
    months = []
    for file in os.listdir('D:/tableau_csvs/trip_data/'):
        if file.startswith(str(x)):
            print(file)
            df = pd.read_csv(f'D:/tableau_csvs/trip_data/{file}')
            df.columns = unique
            months.append(df)

    year_df = pd.concat(months,ignore_index=True,sort=False)
    year_df.to_csv(f'resources/trip_data/output/bulk_{x}.csv',sep=',',index=False)

    print(f'Created bulk CSV for {x}...')

201801-citibike-tripdata.csv
201802-citibike-tripdata.csv
201803-citibike-tripdata.csv
201804-citibike-tripdata.csv
201805-citibike-tripdata.csv
201806-citibike-tripdata.csv
201807-citibike-tripdata.csv
201808-citibike-tripdata.csv
201809-citibike-tripdata.csv
201810-citibike-tripdata.csv
201811-citibike-tripdata.csv
201812-citibike-tripdata.csv
Created bulk CSV for 2018...


### create dictionary of dtypes

In [5]:
dff = pd.read_csv('D:/tableau_csvs/trip_data/201409-citibike-tripdata.csv',nrows=30)
dff.columns.values

array(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid',
       'usertype', 'birth year', 'gender'], dtype=object)

In [6]:
dff.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2828,9/1/2014 00:00:25,9/1/2014 00:47:33,386,Centre St & Worth St,40.714948,-74.002345,450,W 49 St & 8 Ave,40.762272,-73.987882,15941,Subscriber,1980.0,1
1,368,9/1/2014 00:00:28,9/1/2014 00:06:36,387,Centre St & Chambers St,40.712733,-74.004607,2008,Little West St & 1 Pl,40.705693,-74.016777,18962,Subscriber,1982.0,1
2,2201,9/1/2014 00:00:40,9/1/2014 00:37:21,386,Centre St & Worth St,40.714948,-74.002345,441,E 52 St & 2 Ave,40.756014,-73.967416,15982,Subscriber,1968.0,1
3,322,9/1/2014 00:00:41,9/1/2014 00:06:03,167,E 39 St & 3 Ave,40.748901,-73.976049,528,2 Ave & E 31 St,40.742909,-73.977061,19081,Subscriber,1961.0,1
4,1693,9/1/2014 00:00:59,9/1/2014 00:29:12,223,W 13 St & 7 Ave,40.737815,-73.999947,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,20836,Subscriber,1978.0,1


In [7]:
dff.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                 float64
gender                       int64
dtype: object

In [8]:
dtype = {}
apply = ['int64', 'object', 'object', 'int64', 'object', 'float64', 'float64', 'int64', 'object', 'float64', 'float64', 'int64', 'object', 'float64', 'int64']
for x, name in enumerate(dff.columns.values):
    dtype[name] = apply[x]
dtype

{'tripduration': 'int64',
 'starttime': 'object',
 'stoptime': 'object',
 'start station id': 'int64',
 'start station name': 'object',
 'start station latitude': 'float64',
 'start station longitude': 'float64',
 'end station id': 'int64',
 'end station name': 'object',
 'end station latitude': 'float64',
 'end station longitude': 'float64',
 'bikeid': 'int64',
 'usertype': 'object',
 'birth year': 'float64',
 'gender': 'int64'}

### drop nulls and unknowns

In [2]:
import pandas as pd
import os

In [9]:
for file in os.listdir('D:/tableau_csvs/trip_data/output/'):
    if file.startswith('bulk_'):
        print(file)
        df = pd.read_csv(f'D:/tableau_csvs/trip_data/output/{file}',dtype=dtype)
        df2 = df.dropna(how='any')
        df3 = df2[df2.gender != 0]

        print(f'{len(df)} - {len(df2)} - {len(df2) - len(df3)} = {len(df3)}')

        df3.to_csv(f'D:/tableau_csvs/trip_data/output/remove_nulls/{file}',sep=',',index=False)

        print(f'Removed nulls for {file}...')
        print('------------------------------')

bulk_2016.csv
13845655 - 12222976 - 34735 = 12188241
Removed nulls for bulk_2016.csv...
------------------------------
bulk_2017.csv
16364657 - 14718438 - 64318 = 14654120
Removed nulls for bulk_2017.csv...
------------------------------
bulk_2018.csv
17548339 - 17545842 - 1484282 = 16061560
Removed nulls for bulk_2018.csv...
------------------------------
